# Exercise 1
- ### 2.1: What does this tell you about the integrity of the file?
The hash function acts as a "fingerprint" of the file: any modification, even a minor one, would completely change the hash value.Since the hash calculated is the same as the one given in the statement, we can thus conclude that the integrity of the file is maintained.

- ### 2.1: How many bytes do you expect to be affected by this change?
When you change the first 4 bytes of the PDF, the resulting hash will be completely different from the original. This is because the SHA-256 algorithm processes in blocks and each block influences the following ones.Thus, all the bytes of the hash change, even if only 4 bytes of the file have been modified.

# Exercise 2

- ### 2.1: Can you retrieve the passwords?
Yes - the provided `crack_hash.py` successfully recovers the original passwords for both the unsalted and the 1-byte shared-salt scenarios:
- Unsalted: precompute the 20 SHA-256 hashes of `hex_passwds` and map them to the shuffled list (`mixed_hlist`) to recover the passwords.
- Shared 1-byte salt: brute-force the 256 possible salt values; for each candidate salt compute the 20 hashes and compare against `mixed_shlist`. Once the salt is found, reconstruct the original salted password values in the shuffled order.
```python
# --------------------------- START OF EXERCISE 1 ---------------------------

# build a map from hash -> original hex-encoded password bytes
hash_to_pwd = {}
for pwd in hex_passwds:
	digest = hashes.Hash(hashes.SHA256())
	digest.update(pwd)
	phash = hexlify(digest.finalize())
	hash_to_pwd[phash] = pwd

# reconstruct in the shuffled order
for hash in mixed_hlist:
	cracked_pwds.append(hash_to_pwd[hash])

# --------------------------- END OF EXERCISE 1 ---------------------------

# ------------------------------- START OF EXERCISE 2 -------------------------------

# brute force the 1-byte salt
target_set = set(mixed_shlist.tolist() if hasattr(mixed_shlist, 'tolist') else mixed_shlist)
salt_found = None
hash_to_salted_pwd = None

for s in range(256):
	candidate_salt = bytes([s])
	local_map = {}
	local_hashes = []
	for pwd in hex_passwds:
		_digest = hashes.Hash(hashes.SHA256())
		_digest.update(candidate_salt + pwd)
		h = hexlify(_digest.finalize())
		local_hashes.append(h)
		local_map[h] = candidate_salt + pwd
	
	# compare
	if set(local_hashes) == target_set:
		salt_found = candidate_salt
		hash_to_salted_pwd = local_map
		break

# if salt found, reconstruct in shuffled order
if hash_to_salted_pwd is not None:
	for h in mixed_shlist:
		cracked_spwds.append(hash_to_salted_pwd[h])

# ------------------------------- END OF EXERCISE 2 -------------------------------
```
- ### 2.2: How long would the following attacks take?

- **No salt**: trivial - precompute the small set of candidate hashes once and match; **O(N)** work. `20` hashes.  
- **Tiny salt (1 byte)**: still trivial - only 256× increase; **O(256 × N) ≈ O(N)** = tiny and easily brute-forcible. `256 × 20 = 5,120` hashes.  
- **Large salt (8 bytes, same for all)**: brute-forcing the salt space (2^64) is computationally unfeasible. If the **same 8-byte salt** is used across all hashes, an attacker must perform **O(2^k × N)** work (where *k = 64* bits of salt) once to recover all passwords - still impractical. `2^64 × 20 ≈ 3.689×10^20` hashes.  
- **Large salt (8 bytes, unique per hash)**: the attacker must repeat that enormous search per target, multiplying the cost by the number of hashes (here 20), making it even more impractical. **O(N × 2^k × N) = O(N² × 2^k)**. `20 × (2^64 × 20) = 400 × 2^64 ≈ 7.379×10^22` hashes.


# Exercise 3

- ### Check out the SHAttered paper and explain how the attack works.
By calculating the SHA-1 hash of the a.pdf and b.pdf files, it was found that the value is exactly the same for both. However, the files have different contents.This collision is possible due to the SHAttered attack, which found a flaw in the SHA-1 algorithm, breaking its collision resistance.This demonstrates that SHA-1 is no longer safe for verifying the integrity of files, as it is possible to create two different documents with the same "fingerprint".

PATH:~$ openssl dgst -sha1 a.pdf
SHA1(a.pdf)= 5180c7cb22dbdd51e1646498619f65eb25a24582
PATH:~$ openssl dgst -sha1 b.pdf
SHA1(b.pdf)= 5180c7cb22dbdd51e1646498619f65eb25a24582


In [2]:
# Exercise 4

# Download 'hlextend.py' from https://github.com/stephenbradshaw/hlextend to execute a Length Extension Attack
from hlextend import new
import hashlib as hl
import json

##########################
key = os.urandom(27)
message = b"hello"

concat = key + message

# sha2(key || message)
h = hl.sha256(concat)

# Store 'key', 'message' and 'h' in to a file called 'data.json'
data = {
    "key": key.hex(),
    "message": concat.hex(),
    "hash": h.hexdigest()
}
with open('data.json', 'w') as file:
    json.dump(data, file, indent=4)

##########################
# CLEAN PREVIOUS VARIABLES
key = None
message = None
h = None
data = None

##########################
# Get values from data.json file
with open('data.json', 'r') as file:
    data = json.load(file)
    
message = bytes.fromhex(data['message'])
start_hash = data['hash']

# New program reads 'message' and 'h'
extra_message = b'world!'
sha2 = new('sha256')
secret_len = int(16)

# sha2(key || message || extra_message)
print("Message in bytes : ", sha2.extend(extra_message, message, secret_len, start_hash))
print("Message in hexadecimal : ", sha2.hexdigest())

Message in bytes :  b'\xbb\xb3v\x89)\xd3e%\r\xeb\x1d\xe46\x00W\x08A\x0f\x80*\x8ap\xf1.\xca\xb9Thello\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x80world!'
Message in hexadecimal :  cba1d5d38b9e19649b8884a712d42fdce8a704e54adb5f1b929b9d7164296ce4
